In [17]:
import xgboost as xgb
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
train=pd.read_csv('./data/train_1.csv')

def benchmark1(model,testset,label):
    pred=model.predict(testset)
    if pred[pred<0].shape[0]>0:
        print('Neg Value')
    rmse=np.sqrt(mean_squared_error(label,pred))
    lrmse=np.sqrt(mean_squared_error(np.log(label),np.log(pred)))

    print('RMSE:',rmse)
    print('LRMSE:',lrmse)
    return lrmse

In [21]:
def TestDataSet(train):
    y = train['SalePrice']
    train2 = train.drop(['SalePrice'], axis=1)
    X = pd.get_dummies(train2)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=123)
    xg_reg = xgb.XGBRegressor(objective='reg:linear',colsample_bytree=0.7,learning_rate=0.2)
    xg_reg.fit(X_train, y_train)
    benchmark1(xg_reg, X_test, y_test)

TestDataSet(train)

RMSE: 26569.99850416466
LRMSE: 0.11250405472198952


In [11]:
x = train.corr()['SalePrice']

## 去掉极弱相关(|corr| < 0.3)

In [ ]:
TestDataSet(train.drop(x[abs(x)<0.3].index.tolist(), axis=1))

In [1]:
#scale_pos_weight=1, 用来调整正负样本的不平衡, 改变的是learning_rate

In [24]:
#使用降次函数
def TestDataSet2(train):
    y = np.log(train['SalePrice'])
    train2 = train.drop(['Id','SalePrice'], axis=1)
    X = pd.get_dummies(train2)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=123)
    xg_reg = xgb.XGBRegressor(objective='reg:linear',colsample_bytree=0.7,learning_rate=0.01, max_depth=3, n_estimators=3000)
    xg_reg.fit(X_train, y_train)
    benchmark1(xg_reg, X_test, y_test)

TestDataSet2(train)

RMSE: 0.10668233675396087
LRMSE: 0.008955873161179177


In [27]:
#使用降次函数
def TestDataSet3(train):
    y = np.sqrt(train['SalePrice'])
    train2 = train.drop(['Id','SalePrice'], axis=1)
    X = pd.get_dummies(train2)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
    xg_reg = xgb.XGBRegressor(objective='reg:linear',colsample_bytree=0.7,learning_rate=0.01, max_depth=3, n_estimators=3000)
    xg_reg.fit(X_train, y_train)
    #benchmark1(xg_reg, X_test, y_test)
    pred = xg_reg.predict(X_test)
    pred = pred**2
    y_test = y_test**2
    print(np.sqrt(mean_squared_error(np.log(pred), np.log(y_test))))
TestDataSet3(train)

0.10727280106367015


In [28]:
TestDataSet3(train.drop(x[(x<0.1) & (x>0)].index.to_list(), axis=1))

0.1068914479257738


## 小结
1. 适当去掉低相关的列，可能会提升预测的分数--因为去掉噪声；然而也可以去掉更多的信息量
2. 适当变换最终目标的偏态形式，可能回提升预测的分数；然而如果调整的幅度过大，过滤掉更多的信息量
3. 